## Задание

Предложить 2 способа получить таблицу retention 1–6 месяца для когорт 2018-года, с помощью SQL: 
- с использованием оконных функций 
- с использованием операций join. 

В качестве даты заказа используй поле shipped_at таблицы Orders. 

Формат результата: SQL-запросы для подсчета Retention и полученная таблица с метрикой.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

postgres_host = '_'
port = '6432'
database= 'hr-analytics'
user='analytics'
password = '_'

engine = create_engine(
    f'postgresql://{user}:{password}@{postgres_host}:{port}/{database}?sslmode=require'
)

## Запрос с Join

In [ ]:
sql_str = """
WITH 
    fst_ord_dates AS(
        SELECT
            user_id,
            date_trunc('day', MIN(shipped_at)) AS first_order
            FROM orders
        GROUP BY
            user_id
        HAVING date_part('year', MIN(shipped_at)) = 2018
    ),
    
    cohorts as (
    SELECT
        first_order,
        COUNT(user_id) as users_in_cohort
    FROM(
        SELECT  
            date_trunc('day', MIN(shipped_at)) as first_order,
            user_id
        FROM
            orders
        GROUP BY
            user_id
        HAVING
            date_part('year', MIN(shipped_at)) = 2018
        ORDER BY
            first_order ASC
        ) res
    GROUP BY
        first_order
)

SELECT
    first_order,
    users_in_cohort,
    (SUM(sum_1m) / users_in_cohort)::float*100 as retention_1m_perc,
    (SUM(sum_2m) / users_in_cohort)::float*100 as retention_2m_perc,
    (SUM(sum_3m) / users_in_cohort)::float*100 as retention_3m_perc,
    (SUM(sum_4m) / users_in_cohort)::float*100 as retention_4m_perc,
    (SUM(sum_5m) / users_in_cohort)::float*100 as retention_5m_perc,
    (SUM(sum_6m) / users_in_cohort)::float*100 as retention_6m_perc


FROM
    (SELECT
        user_id,
        first_order,
        SUM(DISTINCT month1_flag) as sum_1m,
        SUM(DISTINCT month2_flag) as sum_2m,
        SUM(DISTINCT month3_flag) as sum_3m,
        SUM(DISTINCT month4_flag) as sum_4m,
        SUM(DISTINCT month5_flag) as sum_5m,
        SUM(DISTINCT month6_flag) as sum_6m

    FROM
        (SELECT
            user_id,
            first_order,
            date_trunc('day', shipped_at) as new_order_date,

            MAX(
                CASE WHEN (
                    date_trunc('day', shipped_at) >= first_order + interval '1 month' 
                    AND 
                    date_trunc('day', shipped_at) < first_order + interval '2 month'
                    )
                THEN 1
                ELSE 0
                END
            ) as month1_flag,

            MAX(
            CASE WHEN (
                date_trunc('day', shipped_at) >= first_order + interval '2 month' 
                AND 
                date_trunc('day', shipped_at) < first_order + interval '3 month'
                )
            THEN 1
            ELSE 0
            END
            ) as month2_flag,

            MAX(
            CASE WHEN (
                date_trunc('day', shipped_at) >= first_order + interval '3 month' 
                AND 
                date_trunc('day', shipped_at) < first_order + interval '4 month'
                )
            THEN 1
            ELSE 0
            END
            ) as month3_flag,

            MAX(
            CASE WHEN (
                date_trunc('day', shipped_at) >= first_order + interval '4 month' 
                AND 
                date_trunc('day', shipped_at) < first_order + interval '5 month'
                )
            THEN 1
            ELSE 0
            END
            ) as month4_flag,

            MAX(
            CASE WHEN (
                date_trunc('day', shipped_at) >= first_order + interval '5 month' 
                AND 
                date_trunc('day', shipped_at) < first_order + interval '6 month'
                )
            THEN 1
            ELSE 0
            END
            ) as month5_flag,

            MAX(
            CASE WHEN (
                date_trunc('day', shipped_at) >= first_order + interval '6 month' 
                AND 
                date_trunc('day', shipped_at) < first_order + interval '7 month'
                )
            THEN 1
            ELSE 0
            END
            ) as month6_flag

        FROM
            orders o
            INNER JOIN fst_ord_dates fd
            USING(user_id)
        GROUP BY 
            user_id,
            shipped_at,
            first_order
        ) as agg_stat

    WHERE 
        (month1_flag >= 1) OR 
        (month2_flag >= 1) OR 
        (month3_flag >= 1) OR 
        (month4_flag >= 1) OR 
        (month5_flag >= 1) OR 
        (month6_flag >= 1)
    GROUP BY
        user_id,
        first_order
    ORDER BY
        first_order ASC
    ) active_users_per_months

INNER JOIN cohorts
USING(first_order)

GROUP BY
    first_order,
    users_in_cohort

"""
df_join = pd.read_sql(sql_str,engine)
df_join

,first_order,users_in_cohort,retention_1m_perc,retention_2m_perc,retention_3m_perc,retention_4m_perc,retention_5m_perc,retention_6m_perc
0,2018-08-26,69,42.028986,33.333333,28.985507,30.434783,21.739130,23.188406
1,2018-08-27,296,37.837838,36.824324,28.716216,27.702703,21.621622,22.297297
2,2018-08-28,400,36.500000,31.000000,22.750000,23.750000,19.250000,18.000000
3,2018-08-29,459,37.037037,36.165577,29.847495,27.668845,23.529412,23.529412
4,2018-08-30,390,36.923077,34.358974,26.153846,21.794872,20.512821,20.000000
...,...,...,...,...,...,...,...,...
123,2018-12-27,243,16.049383,10.699588,11.522634,8.641975,12.345679,13.168724
124,2018-12-28,234,13.675214,12.393162,14.102564,9.401709,9.401709,9.829060
125,2018-12-29,258,13.178295,8.527132,10.852713,10.077519,8.914729,8.914729
126,2018-12-30,265,13.962264,13.962264,14.339623,14.339623,13.207547,11.698113


## Запрос с window function

In [ ]:
sql_str = """
    SELECT 
        first_order,
        COUNT(user_id) as users_in_cohort,
        SUM(flag_1m) / COUNT(user_id)::float*100 as retention_1m_perc,
        SUM(flag_2m) / COUNT(user_id)::float*100 as retention_2m_perc,
        SUM(flag_3m) / COUNT(user_id)::float*100 as retention_3m_perc,
        SUM(flag_4m) / COUNT(user_id)::float*100 as retention_4m_perc,
        SUM(flag_5m) / COUNT(user_id)::float*100 as retention_5m_perc,
        SUM(flag_6m) / COUNT(user_id)::float*100 as retention_6m_perc
    FROM
        (SELECT
            first_order,
            user_id,

            MAX(CASE WHEN (
                date_trunc('day', shipped_at) >= first_order + interval '1 month' 
                AND 
                date_trunc('day', shipped_at) < first_order + interval '2 month'
                )
                THEN 1 ELSE 0 END
                ) as flag_1m,

            MAX(CASE WHEN (
                date_trunc('day', shipped_at) >= first_order + interval '2 month' 
                AND 
                date_trunc('day', shipped_at) < first_order + interval '3 month'
                )
                THEN 1 ELSE 0 END
                ) as flag_2m,

            MAX(CASE WHEN (
                date_trunc('day', shipped_at) >= first_order + interval '3 month' 
                AND 
                date_trunc('day', shipped_at) < first_order + interval '4 month'
                )
                THEN 1 ELSE 0 END
                ) as flag_3m,

            MAX(CASE WHEN (
                date_trunc('day', shipped_at) >= first_order + interval '4 month' 
                AND 
                date_trunc('day', shipped_at) < first_order + interval '5 month'
                )
                THEN 1 ELSE 0 END
                ) as flag_4m,

            MAX(CASE WHEN (
                date_trunc('day', shipped_at) >= first_order + interval '5 month'
                AND 
                date_trunc('day', shipped_at) < first_order + interval '6 month'
                )
                THEN 1 ELSE 0 END
                ) as flag_5m,

            MAX(CASE WHEN (
                date_trunc('day', shipped_at) >= first_order + interval '6 month' 
                AND 
                date_trunc('day', shipped_at) < first_order + interval '7 month'
                )
                THEN 1 ELSE 0 END
                ) as flag_6m

        FROM
           (SELECT
                date_trunc('day', shipped_at) as shipped_at,
                first_value(date_trunc('day', shipped_at)) OVER (PARTITION BY user_id) as first_order,
                user_id
            FROM
                orders
            GROUP BY
                user_id,
                shipped_at
            ) as row_stat 
        GROUP BY
            first_order,
            user_id
        HAVING 
            date_part('year', first_order) = 2018
        ORDER BY 
            first_order ASC
        ) as agg_stat
    
    GROUP BY 
        first_order
        
    
"""
df_win_func = pd.read_sql(sql_str, engine)
df_win_func

,first_order,users_in_cohort,retention_1m_perc,retention_2m_perc,retention_3m_perc,retention_4m_perc,retention_5m_perc,retention_6m_perc
0,2018-08-26,69,42.028986,33.333333,28.985507,30.434783,21.739130,23.188406
1,2018-08-27,296,37.837838,36.824324,28.716216,27.702703,21.621622,22.297297
2,2018-08-28,400,36.500000,31.000000,22.750000,23.750000,19.250000,18.000000
3,2018-08-29,459,37.037037,36.165577,29.847495,27.668845,23.529412,23.529412
4,2018-08-30,390,36.923077,34.358974,26.153846,21.794872,20.512821,20.000000
...,...,...,...,...,...,...,...,...
123,2018-12-27,243,16.049383,10.699588,11.522634,8.641975,12.345679,13.168724
124,2018-12-28,234,13.675214,12.393162,14.102564,9.401709,9.401709,9.829060
125,2018-12-29,258,13.178295,8.527132,10.852713,10.077519,8.914729,8.914729
126,2018-12-30,265,13.962264,13.962264,14.339623,14.339623,13.207547,11.698113
